### 1. Model and Tokenizer

In [ ]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

!pip install sentencepiece
!pip install transformers
import transformers
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

In [ ]:
import pickle
with open('tokenizer-gpt2.bin', 'rb') as fp:
  tokenizer = pickle.load(fp)

tokenizer.padding_side = 'left'

# from transformers import set_seed, GPT2Tokenizer
# set_seed(555)

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# tokenizer.padding_side = "left"
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import set_seed, GPT2Config, GPT2Tokenizer
from modeling_gpt2 import GPT2ForSequenceClassification

set_seed(555)

model_config = GPT2Config.from_pretrained(
    'gpt2',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
    num_hidden_layers = 5,
    num_attention_heads = 8,
    hidden_dropout_prob = 0.2,
    attention_probs_dropout_prob = 0.2,
    ignore_mismatched_sizes=True,
    bos_token_id = tokenizer.bos_token_id,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    sep_token_id = tokenizer.sep_token_id
    ) # Binary Classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=model_config)

model.resize_token_embeddings(len(tokenizer))

### 2. Build Dataset

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset

sentiment_to_num = {
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2
}

class TweetDataset(Dataset):
    def __init__(self, train=True):
        super().__init__()
        self.train = train
        self.data = pd.read_csv('sampled_data.csv', sep='\t', encoding='ISO-8859-1')

        df_train = self.data.groupby('sentiment').sample(n=40000).sample(frac=1)
        df_test = self.data.filter(items=list(filter(lambda x: x not in df_train.index, self.data.index)), axis=0)
        df_train = df_train.reset_index()[['text', 'sentiment']]
        df_test = df_test.reset_index()[['text', 'sentiment']]
        df_train['sentiment'] =  df_train['sentiment'].apply(lambda x: sentiment_to_num[x])
        df_test['sentiment'] =  df_test['sentiment'].apply(lambda x: sentiment_to_num[x])

        self.df_train = df_train
        self.df_test = df_test
    
    def __len__(self):
        return len(self.df_train)
    
    def __getitem__(self, index):
        if self.train:
            record = self.df_train.iloc[index]
            text = record['text']
            return {'text': text, 'label': record['sentiment']}
        else:
            record = self.df_test.iloc[index]
            text = record['text']
            return {'text': text, 'label': record['sentiment']}

train_dataset = TweetDataset(train=True)
test_dataset = TweetDataset(train=False)

In [ ]:
for i in range(10):
    print(train_dataset.__getitem__(i)['text'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

with open('/content/drive/MyDrive/data-gpt2.bin', 'wb') as fp:
  pickle.dump([train_dataset, test_dataset], fp)

### 3. Data Collator

In [ ]:
from language_tokens import get_lang_tokens_gpt2

In [ ]:
class Gpt2ClassificationCollator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        
        return
    
    def __call__(self, sequences):
        texts = [sequence['text'] for sequence in sequences]
        labels = [int(sequence['label']) for sequence in sequences]
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding='max_length',
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})

        language_ids = [get_lang_tokens_gpt2(tokenizer.batch_decode(text)) for text in [texts.tolist() for texts in inputs.input_ids]]
        inputs.update({'language_ids': torch.tensor(language_ids)})
        
        return inputs

gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer,
                                                        max_seq_len=60)

### 4. DataLoader

In [ ]:
from torch.utils.data import DataLoader, random_split

train_size = int(len(train_dataset) * 0.8)
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=64,
                              shuffle=True,
                              collate_fn=gpt2classificationcollator)
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=64,
                            shuffle=False,
                            collate_fn=gpt2classificationcollator)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=64,
                             shuffle=False,
                             collate_fn=gpt2classificationcollator)

### 5. Optimizer & Lr Scheduler

In [ ]:
from transformers import AdamW, get_cosine_schedule_with_warmup

total_epochs = 100

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=2e-5,
                  eps=1e-8)

num_train_steps = len(train_dataloader) * total_epochs
num_warmup_steps = int(num_train_steps * 0.1) 

lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
                                               num_warmup_steps=num_warmup_steps,
                                               num_training_steps = num_train_steps)

### 6. Train & Validation

In [ ]:
import torch
from tqdm.auto import tqdm

def train(dataloader, optimizer, scheduler, device_):
    global model
    model.train()
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []

    progress_bar = tqdm(range(len(dataloader)))
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        total_loss.append(loss.item())

        progress_bar.update(1)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevent exploding gradient

        optimizer.step()
        scheduler.step()
        
        prediction_labels += logits.argmax(axis=-1).flatten().tolist()
    
    return true_labels, prediction_labels, total_loss

def validation(dataloader, device_):
    global model
    model.eval()
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []

    progress_bar = tqdm(range(len(dataloader)))
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            logits = logits.detach().cpu().numpy()
            total_loss.append(loss.item())

            progress_bar.update(1)

            prediction_labels += logits.argmax(axis=-1).flatten().tolist()
        
    return true_labels, prediction_labels, total_loss

### 7. Run!

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

all_loss = {'train_loss': [], 'val_loss': []}
all_acc = {'train_acc': [], 'val_acc': []}

for epoch in range(total_epochs):
    y, y_pred, train_loss = train(train_dataloader, optimizer, lr_scheduler, device)
    train_acc = accuracy_score(y, y_pred)
    
    y, y_pred, val_loss = validation(val_dataloader, device)
    val_acc = accuracy_score(y, y_pred)
    
    all_loss['train_loss'] += train_loss
    all_loss['val_loss'] += val_loss
    
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    
    print(f'Epoch: {epoch}, train_loss: {torch.tensor(train_loss).mean():.3f}, train_acc: {train_acc:.3f}, val_loss: {torch.tensor(val_loss).mean():.3f}, val_acc: {val_acc:.3f}') 

    torch.save({
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'losses': all_loss,
        'accuracy': all_acc
    }, '/content/drive/MyDrive/model.bin')
    print('Saved model')

### 7.1. Check Loss with Graph

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

# fig = plt.figure(figsize=(20,20))
# a = fig.add_subplot(4, 1, 1)
# b = fig.add_subplot(4, 1, 2)
# c = fig.add_subplot(2, 1, 2)
# a.plot(all_loss['train_loss'])
# b.plot(all_loss['val_loss'])
# c.plot(all_acc['train_acc'])
# c.plot(all_acc['val_acc'])
# c.set(xlabel='epoch', ylabel='accuracy')
# c.legend(['train', 'val'])

# pass

### 8. Run on Test Data

In [ ]:
# _, y_pred, _ = validation(test_dataloader, device)

# submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
# submit['target'] = y_pred

# submit.to_csv('submission.csv', index=False)